In [1]:
import numpy as np
import xarray as xr

import datetime as dt
import pandas as pd

import matplotlib.pyplot as plt

import altair as alt
alt.data_transformers.enable('json')

from sublimpy import utils
from sublimpy import tidy

In [2]:
file_list = []
for date in ['20221218']:
    print(f"Downloading {date}")
    for hour in range(0,24):
        file = utils.download_sos_highrate_data_hour(
            date = date,
            hour = "{:02d}".format(hour),
            local_download_dir='/Users/elischwat/Development/data/sublimationofsnow/hr_noqc_geo/',
            cache=False
        )
        file_list.append(file)

In [3]:
ds = xr.open_mfdataset(file_list, concat_dim="time", combine="nested")

In [4]:
ls /Users/elischwat/Development/data/sublimationofsnow/hr_noqc_geo/

isfs_geo_hr_20221218_00.nc  isfs_geo_hr_20221218_12.nc
isfs_geo_hr_20221218_01.nc  isfs_geo_hr_20221218_13.nc
isfs_geo_hr_20221218_02.nc  isfs_geo_hr_20221218_14.nc
isfs_geo_hr_20221218_03.nc  isfs_geo_hr_20221218_15.nc
isfs_geo_hr_20221218_04.nc  isfs_geo_hr_20221218_16.nc
isfs_geo_hr_20221218_05.nc  isfs_geo_hr_20221218_17.nc
isfs_geo_hr_20221218_06.nc  isfs_geo_hr_20221218_18.nc
isfs_geo_hr_20221218_07.nc  isfs_geo_hr_20221218_19.nc
isfs_geo_hr_20221218_08.nc  isfs_geo_hr_20221218_20.nc
isfs_geo_hr_20221218_09.nc  isfs_geo_hr_20221218_21.nc
isfs_geo_hr_20221218_10.nc  isfs_geo_hr_20221218_22.nc
isfs_geo_hr_20221218_11.nc  isfs_geo_hr_20221218_23.nc


# Create timestamp
To use the datam, its necessary to combine 3 columns of data from the dataset to get the full timestamp. This is demonstrated below. The 'time' column actually only incudes the second and minute information. For all datapoints, the hour according to the 'time' column is 1.  The 'base_time' column indicates the hour of the day. The 'sample' column indicates the 20hz sample number. 

We demonstrate this in the plots below

In [5]:
df1 = pd.DataFrame({'time': np.unique(ds['time'])})
df2 = pd.DataFrame({'base_time': np.unique(ds['base_time'])})
df3 = pd.DataFrame({'sample': np.unique(ds['sample'])})
(
    alt.Chart(df3).mark_tick(thickness=5).encode(
        alt.X("sample:Q").title(
            f'sample (n = {len(df3)})'
        )
    ).properties(width=600) & 

    alt.Chart(df1).mark_tick(thickness=1).encode(
        alt.X("time:T").axis(
            format='%H%M%p'
        ).title(
            f'time (n = {len(df1)})'
        )
    ).properties(width=600) & 

    alt.Chart(df2).mark_tick(thickness=5).encode(
        alt.X("base_time:T").title(
            f'base_time (n = {len(df2)})'
        )
    ).properties(width=600)
)

alt.VConcatChart(...)

# Get subset of data and turn it into a dataframe

In [6]:
df = ds.to_dataframe().reset_index()

In [7]:
df

,time,sample,base_time,P_0_8m_c,P_0_9m_c,P_1_0m_c,P_1_1m_c,P_10m_c,P_10m_d,P_10m_ue,...,Vpile_c,Vpile_d,Vpile_ue,Vpile_uw,Vpile_Off_d,Vpile_On_d,Vtherm_c,Vtherm_d,Vtherm_ue,Vtherm_uw
0,2022-12-18 00:00:00.500,0,2022-12-18 00:00:00,NaN,716.923157,NaN,NaN,716.148987,716.218018,716.062073,...,NaN,-0.00013,NaN,-0.000131,0.0,0.000445,NaN,1.860211,NaN,1.875406
1,2022-12-18 00:00:00.500,1,2022-12-18 00:00:00,NaN,NaN,NaN,NaN,716.148987,716.218262,716.062256,...,NaN,-0.00013,NaN,-0.000131,0.0,0.000445,NaN,1.860211,NaN,1.875406
2,2022-12-18 00:00:00.500,2,2022-12-18 00:00:00,NaN,NaN,NaN,NaN,NaN,716.217896,716.062378,...,NaN,-0.00013,NaN,-0.000131,0.0,0.000445,NaN,1.860211,NaN,1.875406
3,2022-12-18 00:00:00.500,3,2022-12-18 00:00:00,NaN,NaN,NaN,NaN,716.149170,716.218018,716.062134,...,NaN,-0.00013,NaN,-0.000131,0.0,0.000445,NaN,1.860211,NaN,1.875406
4,2022-12-18 00:00:00.500,4,2022-12-18 00:00:00,NaN,NaN,NaN,NaN,716.148010,716.217957,716.062195,...,NaN,-0.00013,NaN,-0.000131,0.0,0.000445,NaN,1.860211,NaN,1.875406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727995,2022-12-18 00:59:59.500,15,2022-12-18 23:00:00,714.671875,714.573364,713.525269,714.458496,713.813843,713.887207,713.722961,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1727996,2022-12-18 00:59:59.500,16,2022-12-18 23:00:00,714.671753,714.573486,713.524963,714.458740,713.814148,713.887024,713.722656,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1727997,2022-12-18 00:59:59.500,17,2022-12-18 23:00:00,714.671387,714.573608,713.524963,714.458435,713.814087,713.887024,713.723206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1727998,2022-12-18 00:59:59.500,18,2022-12-18 23:00:00,714.671021,714.573242,713.525085,714.458252,713.814087,713.887085,713.723083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create a complete time index

In [8]:
df['time'] = df.apply(lambda row: dt.datetime(
        year = row['time'].year,
        month = row['time'].month,
        day = row['time'].day,
        hour = row['base_time'].hour,
        minute = row['time'].minute,
        second = row['time'].second,
        microsecond = int(row['sample'] * (1e6/20))
    ),
    axis = 1
)

## Add spd variable to dataset 

In [9]:
df['spd_2m_c'] = np.sqrt(df['u_2m_c']**2 + df['v_2m_c']**2)
df['spd_3m_c'] = np.sqrt(df['u_3m_c']**2 + df['v_3m_c']**2)
df['spd_5m_c'] = np.sqrt(df['u_5m_c']**2 + df['v_5m_c']**2)
df['spd_10m_c'] = np.sqrt(df['u_10m_c']**2 + df['v_10m_c']**2)
df['spd_15m_c'] = np.sqrt(df['u_15m_c']**2 + df['v_15m_c']**2)
df['spd_20m_c'] = np.sqrt(df['u_20m_c']**2 + df['v_20m_c']**2)

## Get a subset of the large dataset and make it tidy

In [10]:
df.time.min(), df.time.max()

(Timestamp('2022-12-18 00:00:00'), Timestamp('2022-12-18 23:59:59.950000'))

In [11]:
src = df[
    (df.time.dt.hour == 0)
    & 
    (df.time.dt.minute >= 20)
    & 
    (df.time.dt.hour < 45)
]

In [12]:
src

,time,sample,base_time,P_0_8m_c,P_0_9m_c,P_1_0m_c,P_1_1m_c,P_10m_c,P_10m_d,P_10m_ue,...,Vtherm_c,Vtherm_d,Vtherm_ue,Vtherm_uw,spd_2m_c,spd_3m_c,spd_5m_c,spd_10m_c,spd_15m_c,spd_20m_c
24000,2022-12-18 00:20:00.000,0,2022-12-18,717.071472,717.086365,717.139526,717.044800,716.315002,716.384644,716.228149,...,NaN,1.875285,NaN,NaN,2.007478,1.932025,1.857097,1.699459,1.268717,0.928259
24001,2022-12-18 00:20:00.050,1,2022-12-18,717.071960,717.086548,717.140015,717.044861,716.314941,716.384827,NaN,...,NaN,1.875285,NaN,NaN,2.029325,1.943188,1.876675,1.699459,1.247707,0.917796
24002,2022-12-18 00:20:00.100,2,2022-12-18,717.072021,717.086609,717.140442,717.045044,716.315063,716.384705,NaN,...,NaN,1.875285,NaN,NaN,2.004853,1.880619,1.845559,1.642071,1.248780,0.894764
24003,2022-12-18 00:20:00.150,3,2022-12-18,717.072449,717.086487,717.140930,717.044922,716.315002,716.384644,NaN,...,NaN,1.875285,NaN,NaN,2.021476,1.845177,1.845559,1.623160,1.257478,0.913729
24004,2022-12-18 00:20:00.200,4,2022-12-18,717.072205,717.086487,717.141113,717.045105,716.314636,716.384155,NaN,...,NaN,1.875285,NaN,NaN,2.174189,1.830798,1.766328,1.635002,1.267741,0.908876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71995,2022-12-18 00:59:59.750,15,2022-12-18,716.839539,717.151917,714.626282,716.766357,716.374634,716.443481,716.288513,...,NaN,1.885444,1.866156,NaN,1.543776,1.762632,1.776599,0.998917,0.704424,0.850173
71996,2022-12-18 00:59:59.800,16,2022-12-18,716.840332,717.152100,714.626038,716.766235,716.374817,716.443176,716.288086,...,NaN,1.885444,1.866156,NaN,1.573436,1.706710,1.780226,0.867188,0.690674,0.797129
71997,2022-12-18 00:59:59.850,17,2022-12-18,716.840637,717.151917,714.625854,716.766479,716.374634,716.443237,716.288635,...,NaN,1.885444,1.866156,NaN,1.549683,1.750186,1.729286,0.867188,0.693389,0.703268
71998,2022-12-18 00:59:59.900,18,2022-12-18,716.841187,717.151917,714.625244,716.766541,716.374390,716.443054,716.288086,...,NaN,1.885444,1.866156,NaN,1.550380,1.721081,1.727193,0.628336,0.668968,0.801208


In [13]:
variable_names = [
    'spd_2m_c', 'spd_3m_c', 'spd_5m_c', 'spd_10m_c', 'spd_15m_c', 'spd_20m_c', 
    'u_2m_c', 'u_3m_c', 'u_5m_c', 'u_10m_c', 'u_15m_c', 'u_20m_c', 
    'v_2m_c', 'v_3m_c', 'v_5m_c', 'v_10m_c', 'v_15m_c', 'v_20m_c', 
    'w_2m_c', 'w_3m_c', 'w_5m_c', 'w_10m_c', 'w_15m_c', 'w_20m_c', 
]


# Plot time series of velocity

In [14]:
tidy_src = tidy.get_tidy_dataset(src, variable_names)

OFFSET = 1.5
tidy_src.loc[tidy_src['variable'] == 'spd_3m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'spd_3m_c', 'value'] + 1*OFFSET
tidy_src.loc[tidy_src['variable'] == 'spd_5m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'spd_5m_c', 'value'] + 2*OFFSET
tidy_src.loc[tidy_src['variable'] == 'spd_10m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'spd_10m_c', 'value'] + 3*OFFSET
tidy_src.loc[tidy_src['variable'] == 'spd_15m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'spd_15m_c', 'value'] + 4*OFFSET
tidy_src.loc[tidy_src['variable'] == 'spd_20m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'spd_20m_c', 'value'] + 5*OFFSET

tidy_src.loc[tidy_src['variable'] == 'u_3m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'u_3m_c', 'value'] + 1*OFFSET
tidy_src.loc[tidy_src['variable'] == 'u_5m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'u_5m_c', 'value'] + 2*OFFSET
tidy_src.loc[tidy_src['variable'] == 'u_10m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'u_10m_c', 'value'] + 3*OFFSET
tidy_src.loc[tidy_src['variable'] == 'u_15m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'u_15m_c', 'value'] + 4*OFFSET
tidy_src.loc[tidy_src['variable'] == 'u_20m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'u_20m_c', 'value'] + 5*OFFSET

tidy_src.loc[tidy_src['variable'] == 'v_3m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'v_3m_c', 'value'] + 1*OFFSET
tidy_src.loc[tidy_src['variable'] == 'v_5m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'v_5m_c', 'value'] + 2*OFFSET
tidy_src.loc[tidy_src['variable'] == 'v_10m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'v_10m_c', 'value'] + 3*OFFSET
tidy_src.loc[tidy_src['variable'] == 'v_15m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'v_15m_c', 'value'] + 4*OFFSET
tidy_src.loc[tidy_src['variable'] == 'v_20m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'v_20m_c', 'value'] + 5*OFFSET

tidy_src.loc[tidy_src['variable'] == 'w_3m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'w_3m_c', 'value'] + 1*OFFSET
tidy_src.loc[tidy_src['variable'] == 'w_5m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'w_5m_c', 'value'] + 2*OFFSET
tidy_src.loc[tidy_src['variable'] == 'w_10m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'w_10m_c', 'value'] + 3*OFFSET
tidy_src.loc[tidy_src['variable'] == 'w_15m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'w_15m_c', 'value'] + 4*OFFSET
tidy_src.loc[tidy_src['variable'] == 'w_20m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'w_20m_c', 'value'] + 5*OFFSET

In [15]:
alt.Chart(tidy_src[tidy_src['time'].dt.minute > 55]).transform_filter(
    alt.FieldOneOfPredicate('measurement', [
        # 'u', 'v', 
        'w', 'wind speed'
    ])
).mark_line().encode(
    alt.X("time:T").title("time (utc)"),
    alt.Y("value:Q", title='velocity (m/s)'),
    alt.Color("height:O").scale(scheme='rainbow'),
    alt.Row("measurement:N").sort(['wind speed', 'w'])
).properties(width=600, height=200).display(renderer='svg')

alt.Chart(...)

In [18]:
tidy_src[tidy_src['time'].dt.minute > 55].time.min(), tidy_src[tidy_src['time'].dt.minute > 55].time.max()

(Timestamp('2022-12-18 00:56:00'), Timestamp('2022-12-18 00:59:59.950000'))

In [16]:
rules = alt.Chart(pd.DataFrame({'y':[0,1.5,3,4.5,6,7.5]})).mark_rule().encode(
    alt.Y("y:Q")
)
chart = alt.Chart(tidy_src[tidy_src['time'].dt.minute > 55]).transform_filter(
    alt.datum.measurement == 'w'
).mark_line().encode(
    alt.X("time:T").title("time (utc)"),
    alt.Y("value:Q", title='w (vertical velocity) (m/s)'),
    alt.Color("height:O").scale(scheme='rainbow'),
).properties(width=600, height=300)
(rules + chart).display(renderer='svg')

alt.LayerChart(...)

In [ ]:
alt.Chart(tidy_src[tidy_src['time'].dt.minute > 55]).transform_filter(
    alt.FieldOneOfPredicate('measurement', [
        # 'u', 'v', 
        'w', 'wind speed'
    ])
).mark_line().encode(
    alt.X("time:T").title("time (utc)"),
    alt.Y("value:Q", title='velocity (m/s)'),
    alt.Color("height:O").scale(scheme='rainbow'),
    alt.Column("measurement:N").sort(['wind speed', 'w'])
).properties(width=200, height=200).display(renderer='svg')

In [ ]:
tidy_src = tidy.get_tidy_dataset(src, variable_names)

OFFSET = 0 #1.5
tidy_src.loc[tidy_src['variable'] == 'spd_3m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'spd_3m_c', 'value'] + 1*OFFSET
tidy_src.loc[tidy_src['variable'] == 'spd_5m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'spd_5m_c', 'value'] + 2*OFFSET
tidy_src.loc[tidy_src['variable'] == 'spd_10m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'spd_10m_c', 'value'] + 3*OFFSET
tidy_src.loc[tidy_src['variable'] == 'spd_15m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'spd_15m_c', 'value'] + 4*OFFSET
tidy_src.loc[tidy_src['variable'] == 'spd_20m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'spd_20m_c', 'value'] + 5*OFFSET

tidy_src.loc[tidy_src['variable'] == 'u_3m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'u_3m_c', 'value'] + 1*OFFSET
tidy_src.loc[tidy_src['variable'] == 'u_5m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'u_5m_c', 'value'] + 2*OFFSET
tidy_src.loc[tidy_src['variable'] == 'u_10m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'u_10m_c', 'value'] + 3*OFFSET
tidy_src.loc[tidy_src['variable'] == 'u_15m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'u_15m_c', 'value'] + 4*OFFSET
tidy_src.loc[tidy_src['variable'] == 'u_20m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'u_20m_c', 'value'] + 5*OFFSET

tidy_src.loc[tidy_src['variable'] == 'v_3m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'v_3m_c', 'value'] + 1*OFFSET
tidy_src.loc[tidy_src['variable'] == 'v_5m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'v_5m_c', 'value'] + 2*OFFSET
tidy_src.loc[tidy_src['variable'] == 'v_10m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'v_10m_c', 'value'] + 3*OFFSET
tidy_src.loc[tidy_src['variable'] == 'v_15m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'v_15m_c', 'value'] + 4*OFFSET
tidy_src.loc[tidy_src['variable'] == 'v_20m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'v_20m_c', 'value'] + 5*OFFSET

tidy_src.loc[tidy_src['variable'] == 'w_3m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'w_3m_c', 'value'] + 1*OFFSET
tidy_src.loc[tidy_src['variable'] == 'w_5m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'w_5m_c', 'value'] + 2*OFFSET
tidy_src.loc[tidy_src['variable'] == 'w_10m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'w_10m_c', 'value'] + 3*OFFSET
tidy_src.loc[tidy_src['variable'] == 'w_15m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'w_15m_c', 'value'] + 4*OFFSET
tidy_src.loc[tidy_src['variable'] == 'w_20m_c', 'value'] = tidy_src.loc[tidy_src['variable'] == 'w_20m_c', 'value'] + 5*OFFSET

In [ ]:
alt.Chart(tidy_src[tidy_src['time'].dt.minute > 55]).transform_filter(
    alt.FieldOneOfPredicate('measurement', [
        # 'u', 'v', 
        'w', 'wind speed'
    ])
).mark_line().encode(
    alt.X("time:T").title("time (utc)"),
    alt.Y("value:Q", title='velocity (m/s)'),
    alt.Row("height:O", spacing=0).title("measurement height (m)"),
    alt.Column("measurement:N").sort(['wind speed', 'w'])
).properties(width=200, height=75).display(renderer='svg')

# Calculate Planar Fit

In [ ]:
from sublimpy import extrautils

In [ ]:
(a,b,c), (tilt, tiltaz), W_f = extrautils.calculate_planar_fit(
    ds['u_1m_c'].values.flatten(),
    ds['v_1m_c'].values.flatten(),
    ds['w_1m_c'].values.flatten()
)

In [ ]:
(a,b,c), (np.rad2deg(tilt), np.rad2deg(tiltaz)), W_f

In [ ]:
u, v, w = extrautils.apply_planar_fit(
    ds['u_1m_c'].values.flatten(),
    ds['v_1m_c'].values.flatten(),
    ds['w_1m_c'].values.flatten(),
    a,
    W_f
)

In [ ]:
ds['u_1m_c'].values.flatten(),

In [ ]:
u,v,w